In [ ]:
import numpy as np
from scipy import fftpack
import tensorly as tl
import tensorsketch
from tensorsketch import util
import time
from tensorly.decomposition import tucker
from tensorsketch.tensor_approx import TensorApprox



In [2]:

class SubTensorApprox(TensorApprox): 
    def __init__(self):
        super()
    def st_hosvd(self, over_samp):
        over_samp = 5 
        
        N = len(self.X.shape) 
        
        for mode_n in range(N): 
            tl.unfold(self.X, mode=mode_n)
            
            
        
        
        
        '''
        tucker_core, tucker_factors = tucker(self.X, self.ranks, init='svd')
        X_hat = tl.tucker_to_tensor(tucker_core, tucker_factors)
        running_time = time.time() - start_time
        core_sketch = np.zeros(1)
        arm_sketches = [[] for i in np.arange(len(self.X.shape))]
        sketch_time = -1
        recover_time = running_time
        '''
    
    '''
    sz = [prod(size_tens([1:n-1 n+1:N])) size_core(n)+p(n)];
        Y = tens2mat(Y,n);
        tmp = Y*randn(sz);
        [Q,~] = qr(tmp,0);
        % subspace iteration
        for j = 1:q
            tmp = Y'*Q;
            [Q,~] = qr(tmp, 0);
            tmp = Y*Q;
            [Q,~] = qr(tmp, 0);
        end
        % SVD stage
        tmp = Q'*Y;
        [U{n},S,V] = svd(tmp, 'econ');
        U{n} = Q*U{n};
        sv{n} = diag(S);
        % prepare for next loop
        size_tens(n) = size(U{n},2);
        Y = S*V';
        Y = mat2tens(Y,size_tens,n);
        % compress if needed
        if options.Compress;
            U{n} = U{n}(:, 1:min(size_tens(n), size_core(n)));
        end
    end
    % compute core
    if ~options.Compress, size_core = size_core + p; end
    ind = arrayfun(@(n) 1:n, size_core(1:N), 'UniformOutput', false);
    S = Y(ind{:});
    '''
         
      
        
        Qs = []
        for sketch in self.arm_sketches:
            Q, _ = np.linalg.qr(sketch)
            Qs.append(Q)

        # get the core_(smaller) to implement tucker
        core_tensor = self.X
        N = len(self.X.shape)
        for mode_n in range(N):
            Q = Qs[mode_n]
            core_tensor = tl.tenalg.mode_dot(core_tensor, Q.T, mode=mode_n)
        core_tensor, factors = tucker(core_tensor, ranks=self.ranks)
        self.core_tensor = core_tensor

        # arm[n] = Q.T*factors[n]
        for n in range(len(factors)):
            self.arms.append(np.dot(Qs[n], factors[n]))
        X_hat = tl.tucker_to_tensor(self.core_tensor, self.arms)
        return X_hat, self.arms, self.core_tensor



class TensorApprox(object):
    """
    The wrapper class for approximating the target tensor with three methods: HOOI, two-pass sketching, and one pass sketching
    """

    def __init__(self, X, ranks, ks=[], ss=[], random_seed=1, rm_typ='g', store_phis=True):
        tl.set_backend('numpy')
        self.X = X
        self.ranks = ranks
        self.ks = ks
        self.ss = ss
        self.random_seed = random_seed
        self.rm_typ = rm_typ
        self.store_phis = store_phis

    def run_sim(self):
        X, X0 = tensorsketch.util.square_tensor_gen(self.n, self.rank, dim=self.dim, typ=self.gen_typ, \
                                                    noise_level=self.noise_level, seed=self.random_seed)
        ranks = [self.rank for _ in range(self.dim)]
        ss = [self.s for _ in range(self.dim)]
        ks = [self.k for _ in range(self.dim)]
        tapprox = tensorsketch.tensor_approx.TensorApprox(X, ranks, ks=ks, \
                                                          ss=ss, random_seed=1, rm_typ=self.rm_typ, store_phis=True)
        X_hat_hooi, _, _, _, time_hooi = tapprox.tensor_approx('hooi')
        X_hat_twopass, _, _, _, time_twopass = tapprox.tensor_approx('twopass')
        X_hat_onepass, _, _, _, time_onepass = tapprox.tensor_approx('onepass')
        rerr_hooi = tensorsketch.util.eval_rerr(X, X_hat_hooi, X0)
        rerr_twopass = tensorsketch.util.eval_rerr(X, X_hat_twopass, X0)
        rerr_onepass = tensorsketch.util.eval_rerr(X, X_hat_onepass, X0)
        return (rerr_hooi, rerr_twopass, rerr_onepass), (time_hooi, time_twopass, time_onepass)

    

if method == "hooi":
        tucker_core, tucker_factors = tucker(self.X, self.ranks, init='svd')
        X_hat = tl.tucker_to_tensor(tucker_core, tucker_factors)
        running_time = time.time() - start_time
        core_sketch = np.zeros(1)
        arm_sketches = [[] for i in np.arange(len(self.X.shape))]
        sketch_time = -1
        recover_time = running_time
        
        

class SketchTwoPassRecover(object):
    def __init__(self, X, arm_sketches, ranks):
        tl.set_backend('numpy')
        self.arms = []
        self.core_tensor = None
        self.X = X
        self.arm_sketches = arm_sketches
        self.ranks = ranks

    def recover(self):
        '''
        Obtain the recovered tensor X_hat, core and arm tensor given the sketches
        using the two pass sketching algorithm 
        '''
        # get orthogonal basis for each arm
        Qs = []
        for sketch in self.arm_sketches:
            Q, _ = np.linalg.qr(sketch)
            Qs.append(Q)

        # get the core_(smaller) to implement tucker
        core_tensor = self.X
        N = len(self.X.shape)
        for mode_n in range(N):
            Q = Qs[mode_n]
            core_tensor = tl.tenalg.mode_dot(core_tensor, Q.T, mode=mode_n)
        core_tensor, factors = tucker(core_tensor, ranks=self.ranks)
        self.core_tensor = core_tensor

        # arm[n] = Q.T*factors[n]
        for n in range(len(factors)):
            self.arms.append(np.dot(Qs[n], factors[n]))
        X_hat = tl.tucker_to_tensor(self.core_tensor, self.arms)
        return X_hat, self.arms, self.core_tensor

IndentationError: unexpected indent (<ipython-input-2-231e0f24257f>, line 61)